In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
test_df = pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')

train_df.head()

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
import cv2

class dataset(Dataset) :
    def __init__(self, label, img) :
        self.label = np.array(label)
        img = img / 255.0
        self.img = np.array(img).reshape(-1, 28, 28, 1)
        
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])
        
    def __len__(self) :
        return len(self.img)
    
    def __getitem__(self, index) :
        label = self.label[index]
        img = self.img[index]
        img = self.transform(img)
        
        label = torch.LongTensor([label])
        img = img.float()
        
        return img, label


train_x = train_df.drop(['label'], axis = 1)
train_y = train_df['label']

test_x = test_df.drop(['label'], axis = 1)
test_y = test_df['label']

train_data = dataset(train_y, train_x)
train_data = DataLoader(train_data, batch_size = 64, shuffle = True)

test_data = dataset(test_y, test_x)
test_data = DataLoader(test_data, batch_size = 64, shuffle = True)

In [ ]:
import matplotlib.pyplot as plt

for x, y in train_data :
    plt.imshow(x[0].permute(1, 2, 0))
    plt.show()
    plt.imshow(x[1].permute(1, 2, 0))
    plt.show()
    plt.imshow(x[2].permute(1, 2, 0))
    plt.show()
    break

In [ ]:
#num of labels
classes = 26

In [ ]:
!pip install torchsummary

In [ ]:
import torch.nn as nn
from torchsummary import summary

class cnnModel(nn.Module) :
    def __init__(self, classes) :
        super().__init__()
        self.cnnModel = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1), #14
            nn.LeakyReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1), #7
            nn.LeakyReLU(),
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1), #3
            nn.AdaptiveAvgPool2d((1,1)) #flatten
        )
        
        self.dnnModel = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, classes),
        )
        
    def forward(self, x) :
        output = self.cnnModel(x)
        output = output.squeeze()
        output = self.dnnModel(output)
        return output
    
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = cnnModel(classes).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss().to(device)

summary(model, input_size=(1, 28, 28))

In [ ]:
from tqdm.notebook import tqdm

epoch_num = 100
model.train()

for epoch in range(epoch_num) :
    epoch_loss = 0
    epoch_acc = 0
    for img, label in tqdm(train_data) :
        img = img.to(device)
        label = label.to(device).reshape(-1)
        
        predict = model(img)
        
        loss = criterion(predict, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        correct_prediction = torch.argmax(predict, 1) == label
        correct_prediction = correct_prediction.sum()
        epoch_acc += correct_prediction / img.shape[0]
        
    epoch_loss = epoch_loss / len(train_data)
    epoch_acc = epoch_acc / len(train_data)
    
    print('Epoch : {}/{},   loss : {:.5f},    acc : {:.5f}'.format(epoch+1, epoch_num, epoch_loss, epoch_acc))
    
    if epoch_acc > 0.99 and epoch_loss < 0.1 :
        print('early stop')
        break

In [ ]:
import warnings

warnings.filterwarnings('ignore')
model.eval()

valid_loss = 0
valid_acc = 0
cnt = 0 #show validate result for 5 times

for img, label in tqdm(test_data) :
    img = img.to(device)
    label = label.to(device).reshape(-1)
    
    predict = model(img)
    
    loss = criterion(predict, label)
    
    valid_loss += loss
    correct_prediction = torch.argmax(predict, 1) == label
    correct_prediction = correct_prediction.sum()
    valid_acc += correct_prediction / img.shape[0]
    
    if cnt < 5 :
        fig = plt.figure(figsize = (10, 10))
        for i in range(5) :
            img_i = img[i].to('cpu').numpy()
            img_i = img_i.transpose(1, 2, 0)
            p = torch.argmax(predict[i]).item()
            subplot = fig.add_subplot(1, 5, i+1)
            subplot.set_title(str(label[i].item()) + ' : ' + str(p))
            subplot.imshow(img_i)
        plt.show()
        cnt += 1
        
valid_loss = valid_loss / len(test_data)
valid_acc = valid_acc / len(test_data)

print('loss : {:.5f},    acc : {:.5f}'.format(valid_loss, valid_acc))